# Capstone project battle of the Neighbourhoods

#### By DEVAN CONSTANCE


### Introduction/Business Problem

#### During this time of COVID-19 there has been a drastic shift in both our economies and communities. We have been forced to self-isolate/self-distant and many people have lost their jobs and ultimately had to alter their way of life. However, through these changes there has been an increase in activities that people are able to participate in, one for example is cycling/biking. Though there has been a massive economic hit to many sectors recreational biking has increased substantially where bike shops cannot keep inventory in house, which states that there may be a real opportunity to open a new bike store location. Therefore, in this project I will try to determine a suitable location in Toronto Ontario,  that would best situated to be economically viable, this criteria will be based on the income and population of each neighbourhood and the competitors already located in the surrounding regions.


### Data

#### The data used to obtain the necessary information about population and income will be provided using the census data from https://open.toronto.ca/dataset/neighbourhood-profiles/ - Neighbourhood data 2001,2006,2011.xlsx. The data may be of an archive vintage but, it stores a well maintained and easy to use database of, for ease I used the latest data set and will look to create a comboniation of maps such as a chleoropeth map to visualize the wealthier neighborhoods as well as using the foursquare API to determine the location of the surounding stores in the region to better gauge the best location to build the store in. 


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

using the census data from toronto to collect the income per neighborhood


In [37]:
excel_path='https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/d8d3535e-8b89-4740-9031-5549815e08bc'

toronto_df= pd.read_excel(excel_path,'2011',encoding='latin1')
print('file read')

file read


In [38]:
toronto_df.head()


,Category,Topic,Attribute,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Population,Population,"Population, 2011",2615060.0,30279.0,21988.0,11904.0,29177.0,26918.0,15434.0,19348.0,17671.0,13530.0,23185.0,6488.0,27876.0,21856.0,22057.0,7763.0,14302.0,8713.0,11563.0,17787.0,12053.0,9851.0,10487.0,13093.0,28349.0,24770.0,14612.0,15703.0,13743.0,9444.0,16712.0,26739.0,24363.0,34631.0,34659.0,11449.0,20839.0,14943.0,22829.0,9550.0,22086.0,18810.0,10927.0,22168.0,12474.0,10926.0,31390.0,14083.0,9816.0,11333.0,21292.0,21740.0,13097.0,17656.0,10583.0,12525.0,15853.0,14108.0,13091.0,38084.0,14027.0,10638.0,17058.0,18495.0,21723.0,9170.0,7921.0,44919.0,14642.0,14541.0,15070.0,17011.0,12050.0,9632.0,45086.0,10197.0,10436.0,27167.0,26541.0,17587.0,16306.0,13145.0,32788.0,15982.0,28593.0,10900.0,16423.0,23052.0,21274.0,12191.0,17832.0,13497.0,21073.0,18316.0,9118.0,13746.0,34617.0,8710.0,7653.0,16144.0,11197.0,10007.0,10488.0,22267.0,15050.0,20631.0,45912.0,9632.0,9951.0,16609.0,21251.0,25642.0,17958.0,25017.0,24691.0,27398.0,15594.0,21130.0,10138.0,19225.0,16802.0,7782.0,17182.0,43361.0,26547.0,34100.0,25446.0,18170.0,12010.0,27018.0,45041.0,15004.0,21343.0,53350.0,11703.0,7826.0,13986.0,10578.0,11652.0,27713.0,14687.0
1,Population,Population,"Population, 2006",2503281.0,30156.0,21562.0,11656.0,27482.0,25439.0,14945.0,15325.0,15376.0,13298.0,21961.0,6526.0,25479.0,21652.0,21720.0,7765.0,14198.0,8205.0,11596.0,17318.0,11120.0,10281.0,9815.0,12450.0,24379.0,19850.0,13732.0,14531.0,14405.0,9576.0,16500.0,26644.0,24374.0,34737.0,32010.0,11811.0,20096.0,

now grabbing the neighbourhods from the dataframe and creating a new dataframe for 2006 and 2011 population and income based on their indexed location on the original dataframe

In [70]:
new_df=toronto_df.iloc[[0,689]]
new_df=new_df.drop(['Category','Topic','Attribute','City of Toronto'], axis =1)
new_df

,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,30279.0,21988.0,11904.0,29177.0,26918.0,15434.0,19348.0,17671.0,13530.0,23185.0,6488.0,27876.0,21856.0,22057.0,7763.0,14302.0,8713.0,11563.0,17787.0,12053.0,9851.0,10487.0,13093.0,28349.0,24770.0,14612.0,15703.0,13743.0,9444.0,16712.0,26739.0,24363.0,34631.0,34659.0,11449.0,20839.0,14943.0,22829.0,9550.0,22086.0,18810.0,10927.0,22168.0,12474.0,10926.0,31390.0,14083.0,9816.0,11333.0,21292.0,21740.0,13097.0,17656.0,10583.0,12525.0,15853.0,14108.0,13091.0,38084.0,14027.0,10638.0,17058.0,18495.0,21723.0,9170.0,7921.0,44919.0,14642.0,14541.0,15070.0,17011.0,12050.0,9632.0,45086.0,10197.0,10436.0,27167.0,26541.0,17587.0,16306.0,13145.0,32788.0,15982.0,28593.0,10900.0,16423.0,23052.0,21274.0,12191.0,17832.0,13497.0,21073.0,18316.0,9118.0,13746.0,34617.0,8710.0,7653.0,16144.0,11197.0,10007.0,10488.0,22267.0,15050.0,20631.0,45912.0,9632.0,9951.0,16609.0,21251.0,25642.0,17958.0,25017.0,24691.0,27398.0,15594.0,21130.0,10138.0,19225.0,16802.0,7782.0,17182.0,43361.0,26547.0,34100.0,25446.0,18170.0,12010.0,27018.0,45041.0,15004.0,21343.0,53350.0,11703.0,7826.0,13986.0,10578.0,11652.0,27713.0,14687.0
689,28306.0,29079.0,40376.0,72290.0,62498.0,38422.0,56506.0,49935.0,44023.0,89227.0,30980.0,31007.0,48799.0,24154.0,40610.0,33408.0,213134.0,37582.0,29605.0,58741.0,30736.0,104884.0,49491.0,46104.0,34237.0,45742.0,41386.0,35411.0,44460.0,43619.0,33959.0,29198.0,34337.0,29635.0,33973.0,48746.0,63321.0,28324.0,29596.0,44363.0,43353.0,34153.0,28422.0,71920.0,122460.0,25726.0,41374.0,47713.0,32617.0,50002.0,63148.0,39918.0,38449

we want the neighbourhood to be the index so we will transpose and rename the columns to the appropriate values and sort by the income

In [71]:
new=new_df.transpose()
new= new.rename(columns={0 : "Population 2011"
                         , 689 : "Income 2011"})
new_toronto_df=new.sort_values('Income 2011')
new_toronto_df


,Population 2011,Income 2011
Black Creek,22057.0,24154.0
Oakridge,13497.0,25013.0
Mount Olive-Silverstone-Jamestown,32788.0,25072.0
Glenfield-Jane Heights,31390.0,25726.0
York University Heights,27713.0,26759.0
Humbermede,15853.0,27025.0
Milliken,27167.0,27227.0
Humber Summit,12525.0,27265.0
Thorncliffe Park,19225.0,27345.0
Kennedy Park,17058.0,27622.0
